# Tests of the Light GBM technique

Initial source: https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data preprocessing

In [38]:
dataset = pd.read_csv('../input/Social_Network_Ads.csv')
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [67]:
X = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)
x_train.mean(axis=0)

array([3.71366667e+01, 6.89866667e+04])

In [68]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

sc.get_params()

{'copy': True, 'with_mean': True, 'with_std': True}

In [77]:
x_train = sc.fit_transform(x_train)
print(x_train.mean(axis=0), x_train.std())

[6.66133815e-18 6.29126381e-18] 0.9999999999999999


In [83]:
print(x_test.mean(), x_test.std())

36024.605 43630.07752650658


In [97]:
x_test = sc.transform(x_test)
print(x_test.mean(), x_test.std())

-2.9706749807166357 0.9066650317869929


### LGBM

In [98]:
import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

In [99]:
#Prediction
y_pred=clf.predict(x_test)
#convert into binary values
for i in range(0,100):
    if(y_pred[i] >= 0.5):       # setting threshold to .5
       y_pred[i] = 1
    else:  
       y_pred[i] = 0
#count = int(np.sum(x_test[:,1]) / np.mean(x_test[:,1]))
#count

In [100]:
# Checking results

#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[58,  0],
       [42,  0]], dtype=int64)

In [101]:
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
accuracy

0.58